In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse, parse_qs, unquote
import pandas as pd

url = 'https://g1.globo.com/busca/?q=golpe+digital&order=recent&species=notícias'
aux = 1
news_link_list = []

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

time.sleep(3)
results_div = soup.find('ul', class_='results__list')

if results_div:
    links = [a['href'] for a in results_div.find_all('a', href=True)]
    for link in links:
        news_link_list.append(link)
else:
    print("Div com a classe 'results__list' não encontrada.")

while aux < 30:
    response = requests.get(url+f'&page={aux}')
    soup = BeautifulSoup(response.text, 'html.parser')
    time.sleep(3)
    results_div = soup.find('ul', class_='results__list')

    if results_div:
        links = [a['href'] for a in results_div.find_all('a', href=True)]
        for link in links:
            news_link_list.append(link)
    else:
        print("Div com a classe 'results__list' não encontrada.")
    
    aux = aux+1

In [ ]:
url = 'https://www.cnnbrasil.com.br/tudo-sobre/crimes-ciberneticos/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
aux = 1
cnn_news_list = []

ul = soup.select_one(
    'div.mx-auto.w-full.max-w-\\[1352px\\].px-4.flex.flex-col.gap-x-5.py-2.lg\\:flex-row > '
    'div.w-full.lg\\:w-9\\/12 > div > ul'
)

if ul:
    items = ul.find_all('li')
    for i, li in enumerate(items, 1):
        a_tag = li.find('a')
        if a_tag and a_tag.has_attr('href'):
            cnn_news_list.append(a_tag['href'])
        else:
            print(f"[{i}] Nenhum link encontrado.")
else:
    print("Lista <ul> não encontrada.")

while aux < 10:
    response = requests.get(url+f'pagina/{aux}/')
    soup = BeautifulSoup(response.text, 'html.parser')
    ul = soup.select_one(
        'div.mx-auto.w-full.max-w-\\[1352px\\].px-4.flex.flex-col.gap-x-5.py-2.lg\\:flex-row > '
        'div.w-full.lg\\:w-9\\/12 > div > ul'
    )

    if ul:
        items = ul.find_all('li')
        for i, li in enumerate(items, 1):
            a_tag = li.find('a')
            if a_tag and a_tag.has_attr('href'):
                cnn_news_list.append(a_tag['href'])
            else:
                print(f"[{i}] Nenhum link encontrado.")
    else:
        print("Lista <ul> não encontrada.")
    
    aux = aux+1

In [35]:
combined_news_list = []

for item in news_link_list:
    combined_news_list.append('https:'+item)

for item in cnn_news_list:
    combined_news_list.append(item)

# Pega os textos das noticias

# G1

In [ ]:
url_g1_list = []

for link in news_link_list:
    parsed_url = urlparse(link)
    query_params = parse_qs(parsed_url.query)
    link_real = unquote(query_params.get('u', [''])[0])

    url_g1_list.append(link_real)

In [ ]:
g1_article_texts = []

for link in url_g1_list:
    url = link  # substitua com a URL real
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    article = soup.select_one(
        'div.glb-grid > main > div.mc-article-body > article'
    )

    if article:
        g1_article_texts.append(article.text.strip())
    else:
        print("Elemento <article> não encontrado.")


# CCN

In [ ]:
ccn_article_text = []

for link in cnn_news_list:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    texto_completo = ' '.join(el.get_text(strip=True) for el in soup.select('.break-words'))

    ccn_article_text.append(texto_completo)


# Concatena os textos

In [64]:
final_text_list = []

for item in ccn_article_text:
    final_text_list.append(item)

for item in g1_article_texts:
    final_text_list.append(item)

In [68]:
final_text_df = pd.DataFrame(final_text_list, columns=['text'])

In [70]:
final_text_df.to_csv(r'../src/data/processed/df_text_news.csv',
                    sep=';')